In [ ]:
import pandas as panda
import numpy as num
from scipy.sparse import coo_matrix, vstack, hstack

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
#read preprocessed data from file
train_data = panda.read_csv("train_preprocessed.csv", index_col='id')
print(train_data.shape)
train_data.head()

(256442, 5)


,tid1,tid2,title1_en,title2_en,label
id,,,,,
195611,0,1,two new oldage insurance benefit old people ru...,police disprove bird nest congress person get ...,unrelated
191474,2,3,if come shenzhen sooner later son also come le...,shenzhens gdp outstrips hong kong shenzhen sta...,unrelated
25300,2,4,if come shenzhen sooner later son also come le...,gdp overtopped hong kong shenzhen clarified li...,unrelated
123757,2,8,if come shenzhen sooner later son also come le...,shenzhens gdp overtakes hong kong bureau stati...,unrelated
141761,2,11,if come shenzhen sooner later son also come le...,shenzhens gdp outpaces hong kong defending rum...,unrelated


In [ ]:
#dropping null values from file
train_data = train_data.dropna()
print(train_data.shape)
train_data.head()

(256408, 5)


,tid1,tid2,title1_en,title2_en,label
id,,,,,
195611,0,1,two new oldage insurance benefit old people ru...,police disprove bird nest congress person get ...,unrelated
191474,2,3,if come shenzhen sooner later son also come le...,shenzhens gdp outstrips hong kong shenzhen sta...,unrelated
25300,2,4,if come shenzhen sooner later son also come le...,gdp overtopped hong kong shenzhen clarified li...,unrelated
123757,2,8,if come shenzhen sooner later son also come le...,shenzhens gdp overtakes hong kong bureau stati...,unrelated
141761,2,11,if come shenzhen sooner later son also come le...,shenzhens gdp outpaces hong kong defending rum...,unrelated


In [ ]:
#vectorizing using TF-IDF
title1_vector = TfidfVectorizer(analyzer='word',stop_words= 'english').fit(train_data['title1_en'])
title1_tfidf_vector = title1_vector.transform(train_data['title1_en'])

In [ ]:
title1_tfidf_vector.shape

(256408, 28629)

In [ ]:
#vectorizing title2 column seperately
title2_tfidf_vector = title1_vector.transform(train_data['title2_en'])

In [ ]:
title_stack = hstack([title1_tfidf_vector, title2_tfidf_vector])

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(title_stack, train_data['label'], test_size=0.2)

# Train a Random Forest model on the training set
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)


RandomForestClassifier()

In [ ]:
# Predict the labels of the test data
y_pred = rf.predict(x_test)

# Evaluate the accuracy of the model on the test data
accuracy = rf.score(x_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8588198588198588


In [ ]:
from sklearn import metrics

# Make predictions on the test data
y_pred = rf.predict(x_test)

# Print a classification report
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      agreed       0.85      0.71      0.77     14766
   disagreed       0.80      0.33      0.47      1377
   unrelated       0.86      0.94      0.90     35139

    accuracy                           0.86     51282
   macro avg       0.84      0.66      0.71     51282
weighted avg       0.86      0.86      0.85     51282



In [ ]:
#now applying trained model to predict labels in test data
test_data = panda.read_csv("test_preprocessed.csv", index_col='id')
print(test_data.shape)
test_data.head()

(64110, 4)


,tid1,tid2,title1_en,title2_en
id,,,,
256442,100672,100673,great coat brother zhu zhu wen mandarin love s...,lin xinsheng birth hard milking huo jianhua se...
256443,162269,162270,nasa reveals fact ufo wreckage found moon,ufo found yuancun jiaocheng county shanxi shoc...
256444,157826,157854,hollow tomato loaded hormone,li chenfan bingbing home photo netizen called ...
256445,109579,74076,ange pavilion geoshui accurate matrimony match...,master one eightcharacter presumption marriage...
256446,15068,15085,50yearold busbus blow 8yearold child rumor rum...,joe johnson disgruntled timing order myth


In [ ]:
#dropping null values from test data
test_data = test_data.dropna()
print(test_data.shape)
test_data.head()

(64103, 4)


,tid1,tid2,title1_en,title2_en
id,,,,
256442,100672,100673,great coat brother zhu zhu wen mandarin love s...,lin xinsheng birth hard milking huo jianhua se...
256443,162269,162270,nasa reveals fact ufo wreckage found moon,ufo found yuancun jiaocheng county shanxi shoc...
256444,157826,157854,hollow tomato loaded hormone,li chenfan bingbing home photo netizen called ...
256445,109579,74076,ange pavilion geoshui accurate matrimony match...,master one eightcharacter presumption marriage...
256446,15068,15085,50yearold busbus blow 8yearold child rumor rum...,joe johnson disgruntled timing order myth


In [ ]:
#vectorizing columns in test data
test_title1_tfidf_vector = title1_vector.transform(test_data['title1_en'])
test_title2_tfidf_vector = title1_vector.transform(test_data['title2_en'])
#stacking to form matric with incresed columns
test_title_stack = hstack([test_title1_tfidf_vector, test_title2_tfidf_vector])

In [ ]:
#prediciting labels with the trained model
test_predict_data_labels = rf.predict(test_title_stack)

In [ ]:
#adding labels to test data
test_data['label'] = test_predict_data_labels

In [ ]:
test_data.head()

,tid1,tid2,title1_en,title2_en,label
id,,,,,
256442,100672,100673,great coat brother zhu zhu wen mandarin love s...,lin xinsheng birth hard milking huo jianhua se...,unrelated
256443,162269,162270,nasa reveals fact ufo wreckage found moon,ufo found yuancun jiaocheng county shanxi shoc...,unrelated
256444,157826,157854,hollow tomato loaded hormone,li chenfan bingbing home photo netizen called ...,unrelated
256445,109579,74076,ange pavilion geoshui accurate matrimony match...,master one eightcharacter presumption marriage...,unrelated
256446,15068,15085,50yearold busbus blow 8yearold child rumor rum...,joe johnson disgruntled timing order myth,unrelated


In [ ]:
#extracting label column 
test_label_column = test_data['label']
print(test_label_column)

id
256442    unrelated
256443    unrelated
256444    unrelated
256445    unrelated
256446    unrelated
            ...    
320547       agreed
320548       agreed
320549       agreed
320550    unrelated
320551    unrelated
Name: label, Length: 64103, dtype: object


In [ ]:
#saving output file
from google.colab import files
test_label_column.to_csv('submission.csv')
files.download('submission.csv')